Packet processing speedup demonstration based on vectorization of code.  Initial runs on the whole (>300k packets) capture file took approximately 5 minutes.  The third iteration took approximately 9 seconds, a speedup of 33x.  This demonstrates those iterations using a smaller subsample (1k) of the original data.

In [9]:
import pcapy
from binascii import hexlify
import numpy as np
import time
import matplotlib.pyplot as plt

fn = 'data/2nd_floor_1k.pcap'

%matplotlib inline

In [4]:
# Count number of packets
cap = pcapy.open_offline(fn)

cap.setfilter('dst port 2368')

# get number of matching packets
more_to_read = True
num_packets = 0
while more_to_read:
    try:
        header,packet = cap.next()
        num_packets = num_packets + 1
    except:
        more_to_read=False

The first iteration uses 2 nested loops.  The first loop (i) retreives moves through the packet, looking for 12 laser return reports.  The second loop (j) traverses the set of laser returns, gathering distance and intensity information.

In [5]:
now = time.time()

cap = pcapy.open_offline(fn)
cap.setfilter('dst port 2368')

azimuth = np.zeros(12)

for k in range(num_packets):
    header,packet = cap.next()

    for i in range(12):
        p1 = i*100
        
        block_id = hexlify(packet[p1+42:p1+44][::-1])
        azimuth[i] = float(int(hexlify(packet[p1+44:p1+46][::-1]),16))/100
        for j in range(32):
            p2 = j*3
            
            dist = int(hexlify(packet[p1+p2+46:p1+p2+48][::-1]),16) * 0.002
            intensity = int(hexlify(packet[p1+p2+48][::-1]),16)
          
    gpstime = float(int(hexlify(packet[1242:1246][::-1]),16)) / 10**6
  
print(time.time()-now)  

0.78200006485


The second iteration used numpy's <i>fromString</i> function to change the string data to an integer for each laser block.  In this instance, the outer (i) loop still gathers each block separately, but the (j) loop has been replaced by vectorized code.  This produced a substantial speedup (approximately 5x).

In [6]:
now = time.time()

cap = pcapy.open_offline(fn)
cap.setfilter('dst port 2368')

azimuth = np.zeros(12)

for k in range(num_packets):
    header,packet = cap.next()

    for i in range(12):
        p1 = i*100
        
        block_id = hexlify(packet[p1+42:p1+44][::-1])
        azimuth[i] = float(int(hexlify(packet[p1+44:p1+46][::-1]),16))/100
        
        payload = np.fromstring(packet[p1+46:p1+46+96],dtype=np.uint8).reshape((-1,3))
        distances = 0.002 * (256*(payload[:,1].astype(dtype=float)) + payload[:,0])
        intensities = payload[:,2] 
          
    gpstime = float(int(hexlify(packet[1242:1246][::-1]),16)) / 10**6
  
print(time.time()-now)  

0.159999847412


The third iteration follows the same logic as the second, but the entire codeblock is reshaped as a 12x100 block, making extraction possible with simple indexing and no loops.

In [7]:
now = time.time()

cap = pcapy.open_offline(fn)
cap.setfilter('dst port 2368')

for k in range(num_packets):
    header,packet = cap.next()
    
    payload = np.fromstring(packet[42:1242],dtype=np.uint8).reshape((12,100))
    
    azimuths = 256*payload[:,3] + payload[:,2]
    distances = .002 * (256*payload[:,5::3] + payload[:,4::3])
    intensities = payload[:,6::3]
    gpstime = int(hexlify(packet[1242:1246][::-1]),16)

  
print(time.time()-now)  

0.0329999923706


In [ ]:
You can visualize the regularity in the packet payload:

In [10]:
plt.imshow(payload)